<h1>TITLE</h1>

<h2>Introduction</h2>

<p>In modern video game industry, newsletter is a common make connections to the players. The game company can convey message and post advertisements using the newsletter. Therefore, to increase the profit and the marketing strategies, it is vital to know the characteristics of players who are more willing to subscribe the newsletter.</p>

<p>This project aims to discuss what player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how these features differ between various player types.</p>

<p>R is used in this project to analyze the dataset and summarize them into plots and tables. The dataset used in the project is a comma-separated value (csv) file called <code>player.csv</code>. The dataset includes the following columns:</p>

- experience: The experience of the player (Amateur, Beginner, Pro, Regular, Veteran)
- subscribe: Whether the player subscribed to a game-related newsletter (TRUE, FALSE)
- hashedEmail: The hashed email of the player
- played_hours: The time the player spent at playing the game (in hours)
- name: The player's name
- gender: The player's gender
- Age: The player's age.

<p><b>... !!! DATA DESCRIPTION</b></p>

<h2>Methods & Results</h2>

<p>The purpose of this project is to find out what player characteristics and behaviours can most successfully predict if they subscribed to a game-related newsletter. First of all, in order to analyze and visualize the results using R, some packages are required to be loaded, including <code>tidyverse</code> and <code>tidymodels</code>.</p>

In [ ]:
library(tidyverse)
library(tidymodels)

<p>The function <code>read_csv()</code> can consume the path of a file. We will use it to load the dataset. Let's call the data frame <code>players</code>. We can use the absolute path which always starts with a slash <code>/</code> and lists out the sequence of folders from the root (Timbers et al., 2024). In this case, the absolute path of <code>players.csv</code> is <code>/home/dsci-100-project-yclin9/players.csv</code>.</p>

<p>However, we can use the relative path instead. A relative path only list out the sequence of folders from our current working file. Since the <code>csv</code> file is in the same folder from our current working file, the relative path of the file is <code>players.csv</code></p>

<p>The function <code>head()</code> is designed to look first couple lines of the data frame. Here we can use it to preview the first ten rows of the data frame <code>players</code>.</p>

In [ ]:
players <- read_csv("players.csv")
head(players, 10)

<p>After loading the dataset, the next thing to do is to wrangle and clean the data to the format necessary for the data analysis. Let's name the new data frame <code>players_clean</code>. The <code>mutate()</code> function is used for modifying or adding the columns. For a classification model, the outcome should be a factor rather than a logical. Hence, <code>as_factor()</code> is used to modify the column <code>experience</code> to become logical.</p>

<p>The <code>select()</code> function can help select the columns we need. In this project, we use K-nearest neighbor classification algorithm, which can only predict the result using numeric data (detailed explanation will be given later). Looking at the dataset, only <code>played_hours</code> and <code>Age</code> are numeric data. Therefore, we select <code>played_hours</code>, <code>Age</code>, and <code>subscribe</code> from the dataset.</p>

<p>We except both players' age and played time are predictive of subscribing the game-related newsletter. Young players tend to be more curious and enthusiastic about the game, which makes them have a higher possibility to subscribe for the newsletter. Moreover, people who spend more time on the game shows that they have more interest and willingness to following game-relevant information or update.</p>

<p>Using <code>head()</code> to preview the first ten rows of the data frame, we can see that there are now only needed columns. The variables <code>played_hours</code> and <code>Age</code> are used to evaluate the result and <code>subscribe</code> is the variable we want to label. Additionally, the data type of <code>subscribe</code> is changed from logical (<code>lgl</code>) to factor (<code>dbl</code>).</p>

In [ ]:
players_clean <- players |>
    mutate(subscribe = as_factor(subscribe)) |>
    filter(age, na.rm = TRUE)|>
    select(played_hours, Age, subscribe)
head(players_clean, 10)
players_clean

<p>To visualize the data in R, we have to us the <code>ggplot2</code> package. The <code>ggplot2</code> is already included in the <code>tidyverse</code> metapackage, so no further action is required here to load the package.</p>

<p>We want to produce a scatter plot, with the time players spent on the x-axis and their age on the y-axis, then color the point based on whether they subscribed to a game-related newsletter. We assign those variables in the <code>aes()</code> function. To create a geometric object, we use a <code>geom_*</code> function. In this case, to generate a scatter plot, we use the function <code>geom_point()</code>, the argument <code>alpha</code> makes the dots translucent (Timbers et al., 2024).</code></p>

<p>Since the values of <code>played_hours</code> concentrate between 0 to 1 (hour), the graph will be difficult to read (the first graph presented below). Therefore, <code>scale_x_log10()</code> is used (the second graph presented below) to scale the plot and make it more readable.</p>

<p>As can be seen in the graph, there are more blue dots in the lower half of the graph, at the other hand, dots in the upper half are usually pink. Plus, at the horizontal direction, there are mostly blue dots in the right half of the graph, while there is a mix of blue dots and pink dots in the left half. Those observations indicate that younger players and players are more willing to subscribe to a game-related newsletter. Despite the relation between those variables, more work has to be done to tell how well the age and played time can successfully predict whether or not the player had subscribed to the game-related newsletter.</p>

In [ ]:
players_plot <- ggplot(players_clean, aes(x = played_hours, y = Age, color = subscribe)) +
                    geom_point(alpha = 0.7) +
                    labs(x = "Number of Hours Played", y = "Age",
                         title = "Players' Age vs Number of Hours They Played")

players_plot_scaled <- ggplot(players_clean, aes(x = played_hours, y = Age, color = subscribe)) +
                    geom_point(alpha = 0.7) +
                    labs(x = "Number of Hours Played", y = "Age",
                         title = "Players' Age vs Number of Hours They Played (taken log10)") +
                    scale_x_log10()


players_plot
players_plot_scaled

<p>We are using an algorithm called K-nearest neighbors classification. This method predict the label of the new observation based on the labels of those observations close the new one.</p>

<p>To train a model using K-nearnest neighbor to predict the label of <code>subscribe</code>, we must first use <code>initial_split()</code> to separate the training and testing sets. The argument <code>prop</code> specifies the proportion of the original data set to train, here we take 80% of the original data as the training set. <code>strata</code> is the categorical variable to predict, which is <code>subscribe</code> in the project.</p>

In [ ]:
players_split <- initial_split(players_clean, prop = 0.8, strata = subscribe)
players_training <- training(players_split)
players_testing <- testing(players_split)

<p>The <code>todymodels</code> uses a <code>recipe</code> for all data processing (Timbers et al., 2024). The function <code>recipe()</code> helps create a recipe.</p>

<p>Later on, we must standardize the data to assure that the two variables will influence the results to the same extent. After scaling (<code>step_scale()</code>) and centering (<code>step_center()</code>) all the predictors, they now have a same mean $0$ and standard deviation $1$.</p>

In [ ]:
players_recipe <- recipe(subscribe ~ played_hours + Age, data = players_training) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())
players_recipe

<p>We want to create a model specification, and this requires using the function <code>nearest_neighbor()</code>. <code>weight_func = "rectangular"</code> tells that each of the K nearest neighbors has the same weight when voting (Timbers et al., 2024). We specify the number of <code>neighbors</code> as <code>tune()</code> since we are looking for the best K.</p>

<p>In <code>set_engine()</code>, we select <code>kknn</code> to work on K-nearest neighbor problem. In <code>set_mode()</code>, we specify that we are now performing a classification problem.</p>

In [ ]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

<p>To determine the accuracy of the model, we use a method called cross-validation. We split the training data into $C$ chunks with equal size. One chunk is used as the validation set and the rest are called training set. We calculate the mean of every result to get the accuracy. Here we use 5-fold cross-validation.</p>

In [ ]:
players_vfold <- vfold_cv(players_training, v = 5, strata = subscribe)
players_vfold

<p>We use <code>seq()</code> to create a data frame that has values from 1 to 30, stored by <code>k_val</code>.</p>
<p>Everything required is now prepare. We use the <code>workflow()</code> function to put them together. Using <code>add_recipe()</code> and <code>add_model()</code>, we combined the steps done before. Next, we use <code>tune_grid()</code> to let the model run different values of K (from $1$ to $30$ here).</p>

In [ ]:
k_vals <- tibble(neighbors = seq(from = 1, to = 30, by = 1))

knn_results <- workflow() |>
    add_recipe(players_recipe) |>
    add_model(knn_spec) |>
    tune_grid(resamples = players_vfold, grid = k_vals) |>
    collect_metrics()

accuracies <- knn_results |>
    filter(.metric == "accuracy")

accuracies

<p>Now, after we get the result, the only thing we have to do is to visualize the result. We use the functions in the <code>ggplot2</code> package to produce the plot.</p>

In [ ]:
accuracy_vs_k <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
  geom_point() +
  geom_line() +
  labs(x = "Neighbors", y = "Accuracy Estimate")

accuracy_vs_k

<p>The code below helps sort the results by descending order of mean and take the first result, which will let us get the number of neighbors who has the highest number of mean this time.</p>

In [ ]:
best_k <- accuracies |>
        arrange(desc(mean)) |>
        head(1) |>
        pull(neighbors)
best_k

<p>After getting the value of <code>best_k</code>, we can now evaluate the testing set. The process is mostly same as what was done before, except we changed the values of <code>neighbors</code> from <code>tune()</code> to <code>best_k</code> since we already knew the value.</p>

<p>The function <code>predict()</code> produces a single-column data frame with the predictions of observations in the testing set. Also, we use <code>bind_cols</code> to combined it with the original testing set.</p>

In [ ]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k) |>
  set_engine("kknn") |>
  set_mode("classification")

knn_fit <- workflow() |>
  add_recipe(players_recipe) |>
  add_model(knn_spec) |>
  fit(data = players_testing)

prediction <- predict(knn_fit, players_testing) |>
    bind_cols(players_testing)

prediction

In [ ]:
accuracy <- prediction |>
    accuracy(truth = subscribe, estimate = .pred_class, event_level = "first")
precision <- prediction |>
    precision(truth = subscribe, estimate = .pred_class, event_level = "first")
recall <- prediction |>
    recall(truth = subscribe, estimate = .pred_class, event_level = "first")

accuracy
precision
recall

confusion <- prediction |>
             conf_mat(truth = subscribe, estimate = .pred_class)
confusion

<h2>Discussion</h2>

<p>Based on the results we got, the model has apprximately an accuracy of $???$ of pridicting whether the players subsrribe a game-related newletter using the players' age and time played. This is consistent to our assuption mentioned before that the values of age and played time are somehow predictive to players subsription to the newletter. <b>... !!! more summary about the fidings</b></p>

<p>However, there are some limitations for this project that could not make the results more comprehensive. For instance, the sample size of the data we have is too small, there are only $196$ records, which is too little to build the model. With the lack of enough data, cross-validation or the split of training and testing data may be unstable and inaccurate, especially in our data, the number of players who subscribed and not subscribe is uneven. The performance will vary depending on the folds and splits.</p>

<p>Also, due to the limitations of the algorithm we used, only two variables are compared. Hence, it could not be confirmed that the two chracteristics are the best arguments to use</p>

<p>The topic discussed in this project could lead to many other research questions in the future. For example, we might want to include other variables to make a more comprehensive analysis. The game companies might also want to produce further research about how to attract the players using the newsletter.</p>

<h2>Reference</h2>

<p>Timbers, T., Lee, M., & Campbell, T. (2024, August 26). <i>Data science: A First Introduction.</i> https://datasciencebook.ca/ </p>